### Import Module

In [12]:
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import random
import pickle
import time

### CollectAllProductURL Func

In [2]:
def CollectAllProductURL(searchpage_url, total_page):
    pchome_url = "https://www.pcstore.com.tw"
    result = []
    num_page = 1
    
    while num_page <= total_page:
        time.sleep(random.randint(3,5))
        
        html = requests.get(searchpage_url)
        html.encoding = "big5"
        soup = BeautifulSoup(html.text, 'html.parser')
        
        for td in soup.find_all("td", class_="list_proName"):
            result.append(pchome_url+td.find("a")["href"])
            
        num_page += 1
        searchpage_url = searchpage_url.replace("page={}".format(str(num_page-1)), "page={}".format(str(num_page)))        
    
    return result

### SrapeData Func

In [3]:
def ScrapData(url_lst):
    all_dicts = []
    
    for url in url_lst:
        product_dict = {}
        
        time.sleep(random.randint(3,15))
        html = requests.get(url)
        html.encoding = 'big5'
        soup = BeautifulSoup(html.text, 'html.parser')
        
        # find product title
        try:
            name = soup.find("h1")
            name = name.text
            product_dict["name"] = name
        except:
            print("Exception: {} fails to extract name.".format(url))
            continue
            
        # find product price
        try:
            dollar = soup.find_all("td", class_="t13")
            
            original_price = dollar[0].find("del")
            original_price = original_price.text
            product_dict["original_price"] = original_price
            
            special_price = dollar[1].find("span")
            special_price = special_price.text
            product_dict["special_price"] = special_price
        except:
            print("Exception: {} fails to extract price.".format(url))
            continue
            
        # find imgs & contents: find table
        objective_idx = -1
        for idx,table in enumerate(soup.find_all("table")):
            table = str(table)
            if "技術規格" in table:
                objective_idx = idx
                break
        
        if objective_idx == -1:
            print("Exception: {} fails to find idx.".format(url))
            continue
        
        
        # find imgs & contents: extract table
        try:
            tr = soup.find_all("table")[22].find_all("tr")[3]
        except:
            print("Exception: {} fails to extract table.".format(url))
            continue
            
        
        # find imgs
        try:
            num_img = len(tr.find_all("img"))
            if num_img > 1:
                print("Exception: {} images > 1.".format(url))
                continue
                
            img_url = tr.find("img")["src"]
            product_dict["image"] = img_url
        except:
            print("Exception: {} fails to extract image.".format(url))
            continue
            
            
        # find content 1
        contents = ""
        
        try:
            content1 = tr.find_all("font")
            for string in content1:
                contents += string.text
                contents += "\n"
        except:
            print("Exception: {} fails to extract content1.".format(url))
            continue
        
        # find content 2
        try:
            content2 = tr.find_all("span")
            for string in content2:
                contents += string.text
                contents += "\n"
        except:
            print("Exception: {} fails to extract content2.".format(url))
            continue
            
        product_dict["contents"] = contents
            
        all_dicts.append(product_dict)
        qbar.update(1)
    
    return all_dicts

### Main Func

In [4]:
searchpage_url = "https://www.pcstore.com.tw/obravo/HM/search.htm?st_sort=2&s_page=1&pql_word=IhbfGu6..GHHNoeq-TIge8gse2Iw25OLxGCDaogLjHAe3Oeig2UjUzXse2Iw25OLxG3kk"
all_products_url = CollectAllProductURL(searchpage_url, 3)

In [6]:
qbar = tqdm(total=len(all_products_url))
all_dicts = ScrapData(all_products_url)

Exception: https://www.pcstore.com.tw/obravo/M55108756.htm fails to extract table.
Exception: https://www.pcstore.com.tw/obravo/M55018834.htm images > 1.
Exception: https://www.pcstore.com.tw/obravo/M55040523.htm images > 1.
Exception: https://www.pcstore.com.tw/obravo/M08547616.htm fails to find idx.
Exception: https://www.pcstore.com.tw/obravo/M08547417.htm fails to find idx.
Exception: https://www.pcstore.com.tw/obravo/M24223886.htm fails to find idx.
Exception: https://www.pcstore.com.tw/obravo/M24224968.htm images > 1.


### Data Manipulation

In [9]:
with open("StoreData.pickle","wb") as file:
    pickle.dump(all_dicts, file)

In [15]:
df = pd.DataFrame(all_dicts)
df = df.reindex(["name", "original_price", "special_price", "image", "contents"], axis=1)

In [17]:
df.columns = ["商品名稱", "原價", "特價", "圖片", "描述"]

In [19]:
df.to_excel("whai.xlsx")